# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
M. Flock  ->  M. Flock  |  ['M. Flock']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
J. Li  ->  J. Li  |  ['J. Li']
B. C. Estrada  ->  B. C. Estrada  |  ['B. C. Estrada']
Arxiv has 53 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2501.07627


extracting tarball to tmp_2501.07627...

 done.


/tmp/ipykernel_3195/2822249172.py:52: LatexWarning: 2501.07627 did not run properly
bad escape \i at position 38
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2501.08015/main_resubmitted_forastroph_nolinenumbers.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authorsOKfinal_4july2024.tex' from 'tmp_2501.08015/authorsOKfinal_4july2024.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2501.08301...

 done.


B. C. Estrada  ->  B. C. Estrada  |  ['B. C. Estrada']


Found 73 bibliographic references in tmp_2501.08301/main.bbl.
Error retrieving bib data for bodman2018inferring: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.07633-b31b1b.svg)](https://arxiv.org/abs/2501.07633) | **The coexistence of the streaming instability and the vertical shear instability in protoplanetary disks: Scale-dependence of dust diffusion**  |
|| U. Schäfer, A. Johansen, <mark>M. Flock</mark> |
|*Appeared on*| *2025-01-15*|
|*Comments*| *15 pages, 8 figures, 2 tables; accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            The vertical shear instability and the streaming instability are two robust sources of turbulence in protoplanetary disks. The former has been found to induce anisotropic turbulence that is stronger in the vertical than in the radial dimension and to be overall stronger compared to the largely isotropic turbulence caused by the streaming instability. In this study, we shed light on the dust diffusion by the vertical shear instability and the streaming instability separately and together, and in particular on the direction- and scale-dependence of the diffusion. To this end, we employ two-dimensional global models of the two instabilities either in isolation or in combination. The vertical shear instability in isolation diffuses dust more strongly in the vertical direction than the streaming instability in isolation, resulting in a wave-shaped dust layer in our two-dimensional simulations. Compared with this large-scale diffusion, though, our study highlights that the vertical shear instability causes substantially weaker or even negligible small-scale diffusion. We validate this result using previously published three-dimensional simulations. In particular when simulating centimetre-sized dust, the undulating dust layer becomes internally razor-thin. In contrast, the diffusion owing to the streaming instability exhibits only a marginal scale-dependence, with the dust layer possessing a Gaussian shape. In models including both instabilities, the undulating mid-plane layer is broadened to a width set by the intrinsic diffusion level caused by the streaming instability.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.07637-b31b1b.svg)](https://arxiv.org/abs/2501.07637) | **NuSTAR observations of a varying-flux quasar in the Epoch of Reionization**  |
|| L. Marcotulli, et al. -- incl., <mark>E. Bañados</mark>, <mark>S. Belladitta</mark>, <mark>Y. Khusanova</mark> |
|*Appeared on*| *2025-01-15*|
|*Comments*| *16 pages, 7 Figures, 3 Tables; accepted for publication in the Astrophysical Journal Letters (ApJL)*|
|**Abstract**|            With enough X-ray flux to be detected in a 160s scan by SRG/eROSITA, the $z = 6.19$ quasar CFHQS J142952+544717 is, by far, the most luminous X-ray source known at $z > 6$. We present deep (245 ks) NuSTAR observations of this source; with $\sim180$ net counts in the combined observations, CFHQS J142952+544717 is the most distant object ever observed by the observatory. Fortuitously, this source was independently observed by Chandra $\sim110$ days earlier, enabling the identification of two nearby (30'' and 45'' away), fainter X-ray sources. We jointly fit both Chandra and NuSTAR observations--self-consistently including interloper sources--and find that, to greater than 90% confidence, the observed 3-7 keV flux varied by a factor of $\sim2.6$ during that period, corresponding to approximately two weeks in the quasar rest-frame. This brightening is one the most extreme instances of statistically significant X-ray variability seen in the Epoch of Reionization. We discuss possible scenarios that could produce such rapid change, including X-ray emission from jets too faint at radio frequencies to be observed.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.08301-b31b1b.svg)](https://arxiv.org/abs/2501.08301) | **A Disintegrating Rocky World Shrouded in Dust and Gas: Mid-IR Observations of K2-22b using JWST**  |
|| N. Tusay, et al. -- incl., <mark>B. C. Estrada</mark> |
|*Appeared on*| *2025-01-15*|
|*Comments*| **|
|**Abstract**|            The disintegrating ultra-short period rocky exoplanet K2-22b periodically emits dusty clouds in a dynamically chaotic process resulting in a variable transit depth from 0-1.3%. The effluents that sublimate off the surface and condense out in space are probably representative of the formerly interior layers convectively transported to the molten surface. Transmission spectroscopy of these transiting clouds reveal spectral fingerprints of the interior composition of this rocky world. We used JWST's Mid-Infrared Instrument (MIRI) as a low-resolution slitless spectrograph to observe four predicted transit windows for K2-22b. For each observation, we extracted a transmission spectrum over the spectral range of 4.3-11.8 $\mu$m. We detect one transit at high significance and two at low significance. We find that the data 1) disfavor featureless, iron-dominated core material, 2) are consistent with some form of magnesium silicate minerals, likely from mantle material, and 3) show a distinct and unexpected feature at $\sim$5 $\mu$m. The unexpected feature, also seen weakly in the low-significance transits, is consistent with some gas features, possibly NO and/or CO$_2$. These findings warrant further study to improve the constraints on the composition of this disintegrating rocky world.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.08015-b31b1b.svg)](https://arxiv.org/abs/2501.08015) | **Periodic Gamma-ray Modulation of the blazar PG 1553+113 Confirmed by Fermi-LAT and Multi-wavelength Observations**  |
|| S. Abdollahi, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-01-15*|
|*Comments*| *23 pages, 14 figures, 20 plots, 1 table, accepted and published by The Astrophysical Journal. Article produced by the Fermi Large Area Telescope (LAT) Collaboration*|
|**Abstract**|            A 2.1-year periodic oscillation of the gamma-ray flux from the blazar PG 1553+113 has previously been tentatively identified in almost 7 year of data from the Fermi Large Area Telescope. After 15 years of Fermi sky-survey observations, doubling the total time range, we report >7 cycle gamma-ray modulation with an estimated significance of 4 sigma against stochastic red noise. Independent determinations of oscillation period and phase in the earlier and the new data are in close agreement (chance probability <0.01). Pulse timing over the full light curve is also consistent with a coherent periodicity. Multiwavelength new data from Swift X-Ray Telescope, Burst Alert Telescope, and UVOT, and from KAIT, Catalina Sky Survey, All-Sky Automated Survey for Supernovae, and Owens Valley Radio Observatory ground-based observatories as well as archival Rossi X-Ray Timing Explorer satellite-All Sky Monitor data, published optical data of Tuorla, and optical historical Harvard plates data are included in our work. Optical and radio light curves show clear correlations with the gamma-ray modulation, possibly with a nonconstant time lag for the radio flux. We interpret the gamma-ray periodicity as possibly arising from a pulsational accretion flow in a sub-parsec binary supermassive black hole system of elevated mass ratio, with orbital modulation of the supplied material and energy in the jet. Other astrophysical scenarios introduced include instabilities, disk and jet precession, rotation or nutation, and perturbations by massive stars or intermediate-mass black holes in polar orbit.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.07627-b31b1b.svg)](https://arxiv.org/abs/2501.07627) | **Discovery of Ancient Globular Cluster Candidates in The Relic, a Quiescent Galaxy at z=2.5**  |
|| K. E. Whitaker, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-01-15*|
|*Comments*| *19 pages, 7 figures, submitted to Astrophysical Journal on January 9, 2025 (comments welcome!)*|
|**Abstract**|            Globular clusters (GCs) are some of the oldest bound structures in the Universe, holding clues to the earliest epochs of star formation and galaxy assembly. However, accurate age measurements of ancient clusters are challenging due to the age-metallicity degeneracy. Here, we report the discovery of 42 compact stellar systems within the 'Relic', a massive, quiescent galaxy at $z=2.53$. The Relic resides in an over-density behind the Abell 2744 cluster, with a prominent tidal tail extending towards two low-mass companions. Using deep data from the UNCOVER/MegaScience JWST Surveys, we find that clusters formed in age intervals ranging from 8 Myr up to $\sim2$ Gyr, suggesting a rich formation history starting at $z\sim10$. While the cluster-based star formation history is broadly consistent with the high past star formation rates derived from the diffuse host galaxy light, one potential discrepancy is a tentative $\sim2-3\times$ higher rate in the cluster population for the past Gyr. Taken together with the spatial distribution and low inferred metallicities of these young-to-intermediate age clusters, we may be seeing direct evidence for the accretion of star clusters in addition to their early in-situ formation. The cluster masses are high, $\sim10^6-10^7~M_{\odot}$, which may explain why we are able to detect them around this likely post-merger galaxy. Overall, the Relic clusters are consistent with being precursors of the most massive present-day GCs. This unique laboratory enables the first connection between long-lived, high-redshift clusters and local stellar populations, offering insights into the early stages of GC evolution and the broader processes of galaxy assembly.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 38</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2501.07633/./figures/density_ratio_VSI_SIafterVSI.png', 'tmp_2501.07633/./figures/density_ratio_refinement_VSI.png', 'tmp_2501.07633/./figures/density_ratio_refinement_SIafterVSI.png', 'tmp_2501.07633/./figures/dust_diffusion_comparison.png', 'tmp_2501.07633/./figures/distance_dispersion_VSI.png', 'tmp_2501.07633/./figures/distance_dispersion_SI.png']
copying  tmp_2501.07633/./figures/density_ratio_VSI_SIafterVSI.png to _build/html/
copying  tmp_2501.07633/./figures/density_ratio_refinement_VSI.png to _build/html/
copying  tmp_2501.07633/./figures/density_ratio_refinement_SIafterVSI.png to _build/html/
copying  tmp_2501.07633/./figures/dust_diffusion_comparison.png to _build/html/
copying  tmp_2501.07633/./figures/distance_dispersion_VSI.png to _build/html/
copying  tmp_2501.07633/./figures/distance_dispersion_SI.png to _build/html/
exported in  _build/html/2501.07633.md
    + _build/html/tmp_2501.07633/./figures/density_ratio_VSI_SIafterVSI.png
    + _build/html/tm

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\dust}{\textrm{d}}$
$\newcommand{\gas}{\textrm{g}}$
$\newcommand{\mm}{\textrm{mm}}$
$\newcommand{\cm}{\textrm{cm}}$
$\newcommand{\mum}{\text{\textmu}\textrm{m}}$
$\newcommand{\au}{\textrm{au}}$
$\newcommand{\yr}{\textrm{yr}}$
$\newcommand{\kyr}{\textrm{kyr}}$
$\newcommand{\St}{\textrm{St}}$
$\newcommand{\K}{\textrm{K}}$
$\newcommand{\g}{\textrm{g}}$</div>



<div id="title">

# The coexistence of the streaming instability and the vertical shear instability in protoplanetary disks: Scale-dependence of dust diffusion

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.07633-b31b1b.svg)](https://arxiv.org/abs/2501.07633)<mark>Appeared on: 2025-01-15</mark> -  _15 pages, 8 figures, 2 tables; accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

U. Schäfer, A. Johansen, <mark>M. Flock</mark>

</div>
<div id="abstract">

**Abstract:** The vertical shear instability and the streaming instability are two robust sources of turbulence in protoplanetary disks. The former has been found to induce anisotropic turbulence that is stronger in the vertical than in the radial dimension and to be overall stronger compared to the largely isotropic turbulence caused by the streaming instability. In this study, we shed light on the dust diffusion by the vertical shear instability and the streaming instability separately and together, and in particular on the direction- and scale-dependence of the diffusion. To this end, we employ two-dimensional global models of the two instabilities either in isolation or in combination. The vertical shear instability in isolation diffuses dust more strongly in the vertical direction than the streaming instability in isolation, resulting in a wave-shaped dust layer in our two-dimensional simulations. Compared with this large-scale diffusion, though, our study highlights that the vertical shear instability causes substantially weaker or even negligible small-scale diffusion. We validate this result using previously published three-dimensional simulations. In particular when simulating centimetre-sized dust, the undulating dust layer becomes internally razor-thin. In contrast, the diffusion owing to the streaming instability exhibits only a marginal scale-dependence, with the dust layer possessing a Gaussian shape. In models including both instabilities, the undulating mid-plane layer is broadened to a width set by the intrinsic diffusion level caused by the streaming instability.

</div>

<div id="div_fig1">

<img src="tmp_2501.07633/./figures/density_ratio_VSI_SIafterVSI.png" alt="Fig2.1" width="33%"/><img src="tmp_2501.07633/./figures/density_ratio_refinement_VSI.png" alt="Fig2.2" width="33%"/><img src="tmp_2501.07633/./figures/density_ratio_refinement_SIafterVSI.png" alt="Fig2.3" width="33%"/>

**Figure 2. -** Ratio of dust $\rho_{$\dust$}$ to gas (volume) density $\rho_{$\gas$}$ as a function of radius $r$ and height $z$, the latter in units of gas scale heights $H_{$\gas$}$(upper panels). Zoom-in on dust-to-gas density ratio $\rho_{$\dust$}/\rho_{$\gas$}$ as a function of radius $r$ and height $z$, with grid structure and dust particles being plotted as black lines and grey dots, respectively (lower panels). Simulations of the vertical shear instability only and of the scenario *SIafterVSI* with a dust size of $3 $\cm$$ are depicted in the upper and left panel as well as lower and right panel, respectively. While the dust is stirred up to a marginally greater scale height  with respect to the mid-plane in the former simulation, the dust layer is much thinner. Its width indeed amounts to less than a grid cell, while the layer in the latter simulation extends over multiple cells both in the radial and in the vertical dimension. (*fig:dust_concentration_VSI_SIafterVSI*)

</div>
<div id="div_fig2">

<img src="tmp_2501.07633/./figures/dust_diffusion_comparison.png" alt="Fig4" width="100%"/>

**Figure 4. -** Dimensionless coefficients of vertical dust diffusion $\delta_z$ calculated from the time-averaged root mean square $z_{\textrm{d,rms}}$(circles) and standard deviation $\sigma_{z_{$\dust$}}$(squares) of the vertical dust particle positions (left panel). Dimensionless radial dust diffusion coefficients $\delta_r$ computed considering all dust particles that are initially located within $\Delta r_{\textrm{init}}=0.1 $\au$$(circles) or $\Delta r_{\textrm{init}}=0.01 $\au$$(squares). In both panels, open and filled symbols represent simulations with a dust size of $3 $\mm$$ and $3 $\cm$$, respectively. Simulations of only the vertical shear instability, only the streaming instability, the scenario *SIwhileVSI*, and the scenario *SIafterVSI* are depicted using blue, orange, green, and purple markers, respectively. We note that the vertical diffusion coefficients in the vertical shear instability simulation with the larger dust size fall outside of the ordinate range and are therefore plotted in an inset with a different ordinate. (*fig:dust_diffusion_comparison*)

</div>
<div id="div_fig3">

<img src="tmp_2501.07633/./figures/distance_dispersion_VSI.png" alt="Fig5.1" width="50%"/><img src="tmp_2501.07633/./figures/distance_dispersion_SI.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Difference between average and initial radial separation $\Delta r$ of dust particle pairs as a function of their initial separation in simulations of the vertical shear instability (left panel) and the streaming instability (right panel) with $3 $\cm$$-sized dust.  We note that the scale of the abscissa is linear between $10^{-2}$ and $-10^{-2}$ and logarithmic otherwise. If their initial separation is less than \mbox{${\sim}0.1 $\au$$}(equivalent to $0.08$ and $1.5$ gas scale heights at the inner and outer radial boundaries), the distance between two particles remains roughly constant if only the vertical shear instability is simulated, while the streaming instability causes the distance to gradually increase with time (differently coloured solid lines). The dashed grey lines mark initial separations of $0.004 $\au$$ and $0.2 $\au$$; we choose these scales to measure small-scale and large-scale radial diffusion induced by the two instabilities individually and jointly. If the initial separation exceeds \mbox{${\sim}0.1 $\au$$}, on the other hand, two particles move closer together with time in both simulations. This is because the inwards drift speed of the particles increases with the radius, and the separation of particles at different radii thus decreases. (*fig:radial_distance_diffusion_VSI_SI*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.07633"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\uat}[2]{\href{http://astrothesaurus.org/uat/#2}{#1 (#2)}}$
$\newcommand{\shortname}{J1429+5447}$
$\newcommand{\longname}{\objectname{CFHQS J142952+544717}}$
$\newcommand{\cii}{[C\ensuremath{ \textsc{ii}}]}$
$\newcommand{\mgii}{Mg\ensuremath{ \textsc{ii}}}$
$\newcommand{\RNum}[1]{\uppercase\expandafter{\romannumeral #1\relax}}$</div>



<div id="title">

# NuSTAR observations of a varying-flux quasar in the Epoch of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.07637-b31b1b.svg)](https://arxiv.org/abs/2501.07637)<mark>Appeared on: 2025-01-15</mark> -  _16 pages, 7 Figures, 3 Tables; accepted for publication in the Astrophysical Journal Letters (ApJL)_

</div>
<div id="authors">

L. Marcotulli, et al. -- incl., <mark>E. Bañados</mark>, <mark>S. Belladitta</mark>, <mark>Y. Khusanova</mark>

</div>
<div id="abstract">

**Abstract:** With enough X-ray flux to be detected in a 160 s scan by _SRG_ /eROSITA, the $z=6.19$ quasar $\longname$ is, by far, the most luminous X-ray source known at $z>6$ . We present deep (245 ks) _NuSTAR_ observations of this source; with ${\sim}180$ net counts in the combined observations, $\longname$ is the most distant object ever observed by the observatory. Fortuitously, this source was independently observed by _Chandra_ ${\sim}110$ days earlier, enabling the identification of two nearby ( $30"$ and $45"$ away), fainter X-ray sources. We jointly fit both _Chandra_ and _NuSTAR_ observations \textemdash self-consistently including interloper sources \textemdash and find that, to greater than 90 \% confidence, the observed $3-7 \rm keV$ flux varied by a factor of $\sim2.6$ during that period, corresponding to approximately two weeks in the quasar rest-frame. This brightening is one the most extreme instances of statistically significant X-ray variability seen in the Epoch of Reionization. We discuss possible scenarios that could produce such rapid change, including X-ray emission from jets too faint at radio frequencies to be observed.

</div>

<div id="div_fig1">

<img src="tmp_2501.07637/./Figures/flux_vs_gamma_j1429_wo_xmm.png" alt="Fig3" width="100%"/>

**Figure 3. -** $3-7 \rm keV$ posterior flux distribution versus the full-band photon index posterior distribution for $\shortname$ obtained with the simultaneous fit of the three _Chandra_ sources and the co-added _ NuSTAR_ FPMAs (details in Section \ref{sec:month-time}). The $1\sigma$ and 90\% contour levels are reported in the plot. There is no overlap between the two contours, which is a further indication that the flux of the source varied in the four months (observed frame) between observations. The photon index distributions span a similar range, indicating that more likely the source only varied in flux. Note that these photon indices are extracted by fitting datasets covering different energy ranges, and hence cannot be compared one-to-one. (*fig:dist_flux*)

</div>
<div id="div_fig2">

<img src="tmp_2501.07637/./Figures/sky_image_plot.png" alt="Fig1" width="100%"/>

**Figure 1. -** _Chandra_(left) and _NuSTAR_(right) X-ray observations of $\shortname$. All panels show the same $2^\prime \times 2^\prime$ region, and the sets of images are smoothed by Gaussian kernels of radius $2^{\prime\prime}$(_Chandra_) and $8^{\prime\prime}$(_NuSTAR_). **Top:**$3-10 \rm keV$ images of the single _Chandra_ observation and the combined _NuSTAR_ observation. Reticles on the _Chandra_ image mark the positions of I-SW and I-NE. A circular aperture of radius $25^{\prime\prime}$ is drawn around the centroid of the _NuSTAR_ flux. **Bottom Left:**_Chandra_ observations in a soft 0.5\textendash3.0 keV band (upper) and broad 0.5\textendash10.0 keV band (lower). The two interloper sources are clearly seen in these two bands, despite not contributing significant flux to the $3-10 \rm keV$ band. **Bottom Right**: _NuSTAR_ 3\textendash10 keV images of $\shortname$ from observation 60701063002 (N02, left) and 60701063004 (N04, right) with FPMA (top) and FPMB (bottom). $\shortname$ is clearly detected in the combined image as well as the individual exposures, albeit less significantly in the FPMB images owing to the higher background.
       (*fig:sky_images*)

</div>
<div id="div_fig3">

<img src="tmp_2501.07637/./Figures/flux_change_probability.png" alt="Fig4" width="100%"/>

**Figure 4. -** ** Left:** Histogram of the difference between the _Chandra_ and _NuSTAR_$3-7 \rm keV$ posterior flux distribution obtained by simultaneously fitting all three _Chandra_ data sets and the _NuSTAR_ co-added FPMA. The mean (dashed vertical line) and 90\% confidence limit (CL, solid vertical lines) of the distributions obtained by a Gaussian fit (solid line) are shown. This histogram highlights that $\shortname$ varied in flux between the _Chandra_ and _NuSTAR_ epoch of observation at $>90\%$ CL. The mean of the distribution indicates that the _NuSTAR_ flux is $\sim2.6$ times higher than the _Chandra_ one. Moreover, only 0.26\% of the distribution has a value $\leq0$, in support of the fact that it is highly unlikely that the source did not vary between the two epochs. **Right:** 90\% distribution of flux offsets observed for $\shortname$(dark blue-green) relative to flux offsets observed for lower-redshift ($z=[1,2]$) AGN in a broadly similar regime of black hole masses ($M_{\rm BH}>10^8$) and Eddington ratios ($\lambda_{\rm Edd} > 0.01$) by \citet[from dark to light pink]{2024MNRAS.531.4524G}. Most of the probability distribution of observed brightening occupies the region where fewer than 5\% of comparison flux variations reside. The rapidity of such a significant flux offset is thus unusual for an AGN. (*fig:flux_var*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.07637"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\NTT}[1]{\textcolor{NTT}{\bf[NTT: #1]}}$
$\newcommand{\PSUAA}{Department of Astronomy and Astrophysics, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{\PSUGS}{Department of Geosciences, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{\CEHW}{Center for Exoplanets and Habitable Worlds, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{\Wisc}{Department of Astronomy, University of Wisconsin-Madison, Madison, WI, USA}$
$\newcommand{\NASA}{NASA Goddard Space Flight Center, Greenbelt, MD, 20771, USA}$
$\newcommand{\ASU}{School of Earth and Space Exploration, Arizona State University, PO Box 876004, Tempe, 85287-6004, Arizona, USA}$
$\newcommand{\IMP}{Astrophysics Group, Imperial College London, Blackett Laboratory, Prince Consort Road, London SW7 2AZ, UK}$
$\newcommand{\Ox}{University of Oxford, Oxford, UK}$
$\newcommand{\MaxP}{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{\Bern}{University of Bern}$
$\newcommand{\Dminn}{Department of Physics and Astronomy, University of Minnesota Duluth, Duluth, Minnesota 55812, USA}$
$\newcommand{\NOIR}{U.S. National Science Foundation National Optical-Infrared Astronomy Research Laboratory, 950 N. Cherry Ave., Tucson, AZ 85719, USA}$
$\newcommand{\columnsep}{5pt}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# A Disintegrating Rocky World Shrouded in Dust and Gas: Mid-IR Observations of K2-22b using JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.08301-b31b1b.svg)](https://arxiv.org/abs/2501.08301)<mark>Appeared on: 2025-01-15</mark> - 

</div>
<div id="authors">

N. Tusay, et al. -- incl., <mark>B. C. Estrada</mark>

</div>
<div id="abstract">

**Abstract:** The disintegrating ultra-short period rocky exoplanet K2-22b periodically emits dusty clouds in a dynamically chaotic process resulting in a variable transit depth from 0--1.3 \% . The effluents that sublimate off the surface and condense out in space are probably representative of the formerly interior layers convectively transported to the molten surface. Transmission spectroscopy of these transiting clouds reveal spectral fingerprints of the interior composition of this rocky world. We used JWST’s Mid-Infrared Instrument (MIRI) as a low-resolution slitless spectrograph to observe four predicted transit windows for K2-22b. For each observation, we extracted a transmission spectrum over the spectral range of 4.3--11.8 $\mu$ m. We detect one transit at high significance and two at low significance. We find that the data 1) disfavor featureless, iron-dominated core material, 2) are consistent with some form of magnesium silicate minerals, likely from mantle material, and 3) show a distinct and unexpected feature at $\sim$ 5 $\mu$ m. The unexpected feature, also seen weakly in the low-significance transits, is consistent with some gas features, possibly NO and/or $CO_2$ . These findings warrant further study to improve the constraints on the composition of this disintegrating rocky world.

</div>

<div id="div_fig1">

<img src="tmp_2501.08301/./lightcurves.png" alt="Fig4" width="100%"/>

**Figure 4. -** MIRI LRS lightcurves for all 4 transit observations listed in Table \ref{tab:obs}. The unbinned data are greyed out in the background. The blue points are binned to a time resolution of 8 minutes. The red line is a 2nd order polynomial fit to transit 4 and scaled to the best fit value in each window. Multiple transit models were fit, including the original _K2_ lightcurve and a boxcar, with no significant difference. The 46 min transit duration calculated by [ and è (2015)]() is included as vertical gray dashed lines for reference. The uncertainties in the data points as reported by \texttt{Eureka!} were inflated to match the actual standard deviation in the baseline flux.
             (*fig:all4LCs*)

</div>
<div id="div_fig2">

<img src="tmp_2501.08301/./T4_spectrum_gasNdust.png" alt="Fig2" width="100%"/>

**Figure 2. -** \justifying Results from JWST Cycle 2 GO Program 3315.
\\\hspace{\textwidth}***Top***: Lightcurve of the second half of the phase curve, collected on April 28, 2024, showing a clear transit at the predicted ephemeris. The unbinned time series data is plotted in grey in the background. The blue points are binned to a time resolution of 8 minutes. The average 46-minute duration drawn from [ and è (2015)]() is shown as vertical dashed grey lines for reference. Since the transit duration incorporating the cloud of material is not precisely known and potentially variable with wavelength, the baseline flux was calculated from data outside the baseline boundaries set at $1.5\times$ the average 46-minute duration, centered at the ephemeris and shown as vertical dashed green lines. The red line is a 2nd order polynomial model fit to the data during transit and set to zero elsewhere. The residuals of the lightcurve model are shown in the bottom panel.
\\\hspace{\textwidth}\ ***Middle***: JWST MIRI spectrum showing a clear detection of a transit from 4.3-7 $\mu$m *except* at 4.8$\mu$m.
The unbinned JWST data for each channel is plotted in grey in the background. The binned JWST spectrum in the foreground has a constant spectral resolution of R=17.
The horizontal red dashed line shows the overall transit depth from the MIRI lightcurve in the top panel. The dotted curves show the spectra models of plausible gas species from the DACE database  ([Grimm, Malik and Kitzmann 2021]()) . The dashed curves show representative combinations of iron-magnesium-silicate species predicted by bodman2018inferring, [Owen, et. al (2024)](). The solid purple line shows a total opacity model representing a combination of these minerals and gases with features that plausibly correspond to features in the data.
CHEOPS photometry is also shown over its 0.33-1.1 $\mu$m bandpass. The measured CHEOPS transit depth is consistent with the overall transit depth from the MIRI lightcurve.
\\\hspace{\textwidth}\ ***Bottom Insets***: The lightcurve and transit models in two channels that illustrate an unexpected spectrum feature with the most significance at $\sim$5 $\mu$m. A flat spectrum is disfavored ($\Delta \chi_\nu^2 = 2.98$, or $3.7\sigma$). The unexpected feature at 5.1$\mu$m, consistent with nitric oxide (NO) gas is clearly detected, in contrast to a clear non-detection at 4.8$\mu$m (see Figure \ref{fig:Ices}).
 (*spectrum*)

</div>
<div id="div_fig3">

<img src="tmp_2501.08301/./4spectra.png" alt="Fig7.1" width="50%"/><img src="tmp_2501.08301/./stacked_all4.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** MIRI spectra during all four expected transits using the 2-degree transit model from transit 4, scaled to the optimal depth in each channel for each transit.
            \\\hspace{\textwidth}*Top:* Individual spectra during each transit window. The red dashed line marks the overall transit depth in T4. The purple dashed line indicates the overall transit depth in each observation as measured from the white-light lightcurves. The white-light time series of transit 4 shows a clear signal, while the others do not (see Figure \ref{fig:all4LCs}).
            However, the most or second most significant detections in T1, T2 and T3 are in the 4.5 and 5.1 $\mu$m channels, supporting the reality of these narrow features observed at modest significance in T4.
            \\\hspace{\textwidth}*Bottom:* Combined spectra of all four transits. The 4.5 and 5.1 $\mu$m features are clearly evident.
             (*fig:4spectra*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.08301"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

310  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

15  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
